In [1]:
import os
import json
import numpy as np
import pandas as pd
import os.path as op
from sklearn.metrics import roc_auc_score

In [2]:
files = [a for a in os.walk("../results/")][0][2]

In [3]:
def get_data(x, f):
    with open(op.join(f, x), "r") as oh:
        return(json.load(oh))

In [4]:
def aurocs(a):
    return(
        [
            roc_auc_score(y_true=a["train.reals"], y_score=a["train.preds"]),
            roc_auc_score(y_true=a["test.reals"], y_score=a["test.preds"])
        ]
    )

In [5]:
ru_1hot = list(
    sorted(
        list(filter(lambda x: "_1hot_" in x and "ru" in x and "json" in x, files)),
        key=lambda x: int(x.split("_")[-1].split(".")[0])
    )
)
en_1hot = list(
    sorted(
        list(filter(lambda x: "_1hot_" in x and "en" in x and "json" in x, files)),
        key=lambda x: int(x.split("_")[-1].split(".")[0])
    )
)
ru_wv = list(
    sorted(
        list(filter(lambda x: "_wv_" in x and "ru" in x and "json" in x, files)),
        key=lambda x: int(x.split("_")[-1].split(".")[0])
    )
)
en_wv = list(
    sorted(
        list(filter(lambda x: "_wv_" in x and "en" in x and "json" in x, files)),
        key=lambda x: int(x.split("_")[-1].split(".")[0])
    )
)
ru_1hot = [get_data(a, "../results/") for a in ru_1hot]
en_1hot = [get_data(a, "../results/") for a in en_1hot]
ru_wv = [get_data(a, "../results/") for a in ru_wv]
en_wv = [get_data(a, "../results/") for a in en_wv]

In [6]:
ru_sham_aucs_1hot = [aurocs(a) for a in ru_1hot[0:23]]
ru_vl_aucs_1hot = [aurocs(a) for a in ru_1hot[24:23+25+1]]
ru_eoff_aucs_1hot = [aurocs(a) for a in ru_1hot[23+25+1:23+25+25]]
ru_eon_aucs_1hot = [aurocs(a) for a in ru_1hot[23+25+25:]]
ru_sham_aucs_wv = [aurocs(a) for a in ru_wv[0:23]]
ru_vl_aucs_wv = [aurocs(a) for a in ru_wv[24:23+25+1]]
ru_eoff_aucs_wv = [aurocs(a) for a in ru_wv[23+25+1:23+25+25]]
ru_eon_aucs_wv = [aurocs(a) for a in ru_wv[23+25+25:]]

In [7]:
en_sham_aucs_1hot = [aurocs(a) for a in en_1hot[0:17]]
en_vl_aucs_1hot = [aurocs(a) for a in en_1hot[17:]]
en_sham_aucs_wv = [aurocs(a) for a in en_wv[0:17]]
en_vl_aucs_wv = [aurocs(a) for a in en_wv[17:]]

In [8]:
ru_1hot_aucs = np.stack([aurocs(a) for a in ru_1hot])
en_1hot_aucs = np.stack([aurocs(a) for a in en_1hot])
ru_wv_aucs = np.stack([aurocs(a) for a in ru_wv])
en_wv_aucs = np.stack([aurocs(a) for a in en_wv])

In [17]:
def get_results(total, sham, vl, eoff, eon):
    return(
        pd.DataFrame(
            {
                "trial": [
                    "all", "sham", "vl", "eoff", "eon"
                ],
                "train_mean": [
                    np.mean(total, 0)[0], np.mean(sham, 0)[0],
                    np.mean(vl, 0)[0], np.mean(eoff, 0)[0],
                    np.mean(eon, 0)[0]
                ],
                "train_std": [
                    np.std(total, 0)[0], np.std(sham, 0)[0],
                    np.std(vl, 0)[0], np.std(eoff, 0)[0],
                    np.std(eon, 0)[0]
                ],
                "test_mean": [
                    np.mean(total, 0)[1], np.mean(sham, 0)[1],
                    np.mean(vl, 0)[1], np.mean(eoff, 0)[1],
                    np.mean(eon, 0)[1]
                ],
                "test_std": [
                    np.std(total, 0)[1], np.std(sham, 0)[1],
                    np.std(vl, 0)[1], np.std(eoff, 0)[1],
                    np.std(eon, 0)[1]
                ]
            }
        )
    )

In [23]:
ru_1hot_results = get_results(
    ru_1hot_aucs, ru_sham_aucs_1hot, ru_vl_aucs_1hot, ru_eoff_aucs_1hot,
    ru_eon_aucs_1hot
)
ru_wv_results = get_results(
    ru_wv_aucs, ru_sham_aucs_wv, ru_vl_aucs_wv, ru_eoff_aucs_wv,
    ru_eon_aucs_wv
)
en_1hot_results = get_results(
    en_1hot_aucs, en_sham_aucs_1hot, en_vl_aucs_1hot, en_vl_aucs_1hot,
    en_sham_aucs_1hot
).iloc[0:3]
en_wv_results = get_results(
    en_wv_aucs, en_sham_aucs_wv, en_vl_aucs_wv, en_vl_aucs_wv,
    en_sham_aucs_wv
).iloc[0:3]

In [25]:
ru_1hot_results

,trial,train_mean,train_std,test_mean,test_std
0,all,0.498695,0.010770,0.501186,0.020455
1,sham,0.497954,0.012057,0.498499,0.015016
2,vl,0.499969,0.004261,0.501074,0.011245
3,eoff,0.497633,0.017426,0.508254,0.029912
4,eon,0.499030,0.004413,0.497479,0.019537


In [30]:
print(ru_1hot_results.to_latex())

\begin{tabular}{llrrrr}
\toprule
{} & trial &  train\_mean &  train\_std &  test\_mean &  test\_std \\
\midrule
0 &   all &    0.498695 &   0.010770 &   0.501186 &  0.020455 \\
1 &  sham &    0.497954 &   0.012057 &   0.498499 &  0.015016 \\
2 &    vl &    0.499969 &   0.004261 &   0.501074 &  0.011245 \\
3 &  eoff &    0.497633 &   0.017426 &   0.508254 &  0.029912 \\
4 &   eon &    0.499030 &   0.004413 &   0.497479 &  0.019537 \\
\bottomrule
\end{tabular}



In [26]:
ru_wv_results

,trial,train_mean,train_std,test_mean,test_std
0,all,1.0,0.0,0.514018,0.069148
1,sham,1.0,0.0,0.515245,0.065297
2,vl,1.0,0.0,0.495266,0.071738
3,eoff,1.0,0.0,0.539783,0.077794
4,eon,1.0,0.0,0.510076,0.055306


In [31]:
print(ru_wv_results.to_latex())

\begin{tabular}{llrrrr}
\toprule
{} & trial &  train\_mean &  train\_std &  test\_mean &  test\_std \\
\midrule
0 &   all &         1.0 &        0.0 &   0.514018 &  0.069148 \\
1 &  sham &         1.0 &        0.0 &   0.515245 &  0.065297 \\
2 &    vl &         1.0 &        0.0 &   0.495266 &  0.071738 \\
3 &  eoff &         1.0 &        0.0 &   0.539783 &  0.077794 \\
4 &   eon &         1.0 &        0.0 &   0.510076 &  0.055306 \\
\bottomrule
\end{tabular}



In [28]:
en_1hot_results

,trial,train_mean,train_std,test_mean,test_std
0,all,0.499254,0.002686,0.501446,0.005701
1,sham,0.499418,0.002330,0.501471,0.005882
2,vl,0.499081,0.003009,0.501420,0.005501


In [28]:
en_1hot_results

,trial,train_mean,train_std,test_mean,test_std
0,all,0.499254,0.002686,0.501446,0.005701
1,sham,0.499418,0.002330,0.501471,0.005882
2,vl,0.499081,0.003009,0.501420,0.005501


In [32]:
print(en_1hot_results.to_latex())

\begin{tabular}{llrrrr}
\toprule
{} & trial &  train\_mean &  train\_std &  test\_mean &  test\_std \\
\midrule
0 &   all &    0.499254 &   0.002686 &   0.501446 &  0.005701 \\
1 &  sham &    0.499418 &   0.002330 &   0.501471 &  0.005882 \\
2 &    vl &    0.499081 &   0.003009 &   0.501420 &  0.005501 \\
\bottomrule
\end{tabular}



In [27]:
en_wv_results

,trial,train_mean,train_std,test_mean,test_std
0,all,1.0,0.0,0.526896,0.076370
1,sham,1.0,0.0,0.503757,0.082510
2,vl,1.0,0.0,0.550036,0.061531


In [33]:
print(en_wv_results.to_latex())

\begin{tabular}{llrrrr}
\toprule
{} & trial &  train\_mean &  train\_std &  test\_mean &  test\_std \\
\midrule
0 &   all &         1.0 &        0.0 &   0.526896 &  0.076370 \\
1 &  sham &         1.0 &        0.0 &   0.503757 &  0.082510 \\
2 &    vl &         1.0 &        0.0 &   0.550036 &  0.061531 \\
\bottomrule
\end{tabular}

